## Best practices for releases

This notebook is a walkthrough of the Cytoscape Release build process. It produces signed installers for Cytoscape, and deploys code to the Nexus server and GitHub.

Use the BUILD_PARENT_DIR variable to specify a directory to build cytoscape in.

#### NOTE: Make sure BUILD_PARENT_DIR is in `.gitignore`

1. Update versions
1. Update Core apps to use new API
1. Build Installers
1. Test on different OSs
1. Deploy subrepos to nexus
1. Post the installers to http://github.com/cytoscape/cytoscape/releases
1. Deploy updated core apps

After release:
1. Merge the branch used to build installers into master and tag the release
1. If master release, update the version in dev branch
1. Update Cytoscape manual via https://github.com/cytoscape/cytoscape-manual
1. Update www.github.com/cytoscape/cytoscape.github.com (downloads page, roadmap, etc.)
1. Update system requirements script at https://github.com/cytoscape/cytoscape-admin-scripts/tree/master/system-checker

Before starting this process, it is a good idea to Restart and Clear Output for this notebook's Kernel.

### 1. Set up Build Environment

This sets up the build environment, including Java and Maven versions, and the root directory of the build.


In [1]:
from subprocess import Popen, PIPE
import os
import shutil

Set the notebook directory. Note that this is system dependent.

In [2]:
NOTEBOOK_DIR = '/home/cybuilder/cytoscape-admin-scripts'

print(NOTEBOOK_DIR)

/home/cybuilder/cytoscape-admin-scripts


Set the Java environment. This is Java 11 for Cytoscape 3.8.0 and above.

Note that this is system dependent, and may need to be changed to reflect the system's configuration.

In [3]:
%env JAVA_HOME=/usr/lib/jvm/jre-11-openjdk

env: JAVA_HOME=/usr/lib/jvm/jre-11-openjdk


Set the MAVEN_HOME environment variable, as well as point the path to the correct Maven binaries. 

Note that this is system dependent and may need to be changed to reflect the system's configuration.

In [4]:
%env MAVEN_HOME=/opt/maven
%env PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin

env: MAVEN_HOME=/opt/maven
env: PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


Prepare the target directory and clone the cytoscape git repo.

In [5]:
print('Changing to directory: ' + NOTEBOOK_DIR)
os.chdir(NOTEBOOK_DIR)

# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), 'install4j8_build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)
else:
    shutil.rmtree(BUILD_PARENT_DIR)
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Changing to directory: /home/cybuilder/cytoscape-admin-scripts
Cloning into 'cytoscape'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 568 (delta 20), reused 20 (delta 7), pack-reused 529
Receiving objects: 100% (568/568), 767.17 KiB | 0 bytes/s, done.
Resolving deltas: 100% (273/273), done.


In [6]:
STARTING_BRANCH = 'release/3.8.1'     # develop for major release, release/3.X.X for minor release

### 2. Pull the release/3.8.1 branch of Cytoscape

Note that to execute the checkout step here, you will need to have set up an SSH key on this machine that does not use password validation, or else parts of the build will be stall when they require input. 

In [7]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

Target directory = 
Cytoscape project will be cloned to: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape
Cloning into 'cytoscape'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 568 (delta 20), reused 20 (delta 7), pack-reused 529
Receiving objects: 100% (568/568), 767.17 KiB | 0 bytes/s, done.
Resolving deltas: 100% (273/273), done.
Cloning: parent (URI = git@github.com:cytoscape/cytoscape-parent.git)
Cloning into 'parent'...
remote: Enumerating objects: 1640, done.
remote: Total 1640 (delta 0), reused 0 (delta 0), pack-reused 1640
Receiving objects: 100% (1640/1640), 255.12 KiB | 0 bytes/s, done.
Resolving deltas: 100% (628/628), done.
~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/parent ~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape
Already on 'develop'
~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape
Cloning: api (U

## 2a. Reset

In [8]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

------------------------------------------------------------------------
Executing command: git clean -f -d
--in .
------------------------------------------------------------------------
--in parent
------------------------------------------------------------------------
--in api
------------------------------------------------------------------------
--in impl
------------------------------------------------------------------------
--in support
------------------------------------------------------------------------
--in gui-distribution
------------------------------------------------------------------------
--in app-developer
------------------------------------------------------------------------
------------------------------------------------------------------------
Executing command: git reset --hard
--in .
HEAD is now at aed26c0 Branch for 3.8.1
------------------------------------------------------------------------
--in parent
HEAD is now at 46ff74e Branch for 3.8.1
--------

## 2. Verify status

In [28]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

------------------------------------------------------------------------
Executing command: git status
--in .
# On branch release/3.8.1
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   build_output.txt
#
no changes added to commit (use "git add" and/or "git commit -a")
------------------------------------------------------------------------
--in parent
# On branch release/3.8.1
nothing to commit, working directory clean
------------------------------------------------------------------------
--in api
# On branch release/3.8.1
nothing to commit, working directory clean
------------------------------------------------------------------------
--in impl
# On branch feature/3.8.1_fixes_for_large_files
nothing to commit, working directory clean
------------------------------------------------------------------------
--in support
# On branch release/3.8.

## 3. Build Cytoscape and ensure no errors

This may take a while. Expect to build subrepos first before building from the root directory

Currently, you should expect two or so non-fatal javadoc-bundle-options related errors.

In [29]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn clean install -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

Showing ERROR lines in build...
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/app-api/target/javadoc-bundle-options. Ignored it.
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/swing-app-api/target/javadoc-bundle-options. Ignored it.


## 4. Build Cytoscape installers

This requires Install4J to be configured on your machine and for the Install4j project file to point to the correct Mac signing file. The keystore password is set below (blank in this config), to be interpreted by the Install4j Maven plugin.

In [30]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
%env MAC_KEYSTORE_PASSWORD=
!mvn clean install -U

env: MAC_KEYSTORE_PASSWORD=
[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape.distribution:packaging:jar:3.8.1
[WARNING] 'build.plugins.plugin.version' for org.sonatype.install4j:install4j-maven-plugin is missing. @ org.cytoscape.distribution:packaging:[unknown-version], /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/pom.xml, line 104, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ----------------< org.cytoscape.distribution:packaging >----------------
[INFO] Building Cytoscape Release Packaging 3.8.1
[INFO] --------------------------------[ jar ]---------------------------------
Downloaded from central: https://repo.mave

[INFO] Loading config file /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/classes/Cytoscape.install4j
[INFO] Creating media file 'Windows 64 bit':
[INFO] Created media file output directory /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/media
[INFO] 
[INFO]   Collecting files:
[INFO]   Compiling launchers:
[INFO]     Compiling launcher 'Cytoscape':
[INFO]   Creating media file: 
[INFO]     Zipping custom code & resources JAR file
[INFO]     Identifying components
[INFO]     Shrinking runtime
[INFO]   
[INFO] Creating media file 'Windows 32 bit':
[INFO]   Collecting files:
[INFO]   Compiling launchers:
[INFO]     Compiling launcher 'Cytoscape':
[INFO]   Creating media file: 
[INFO]     Zipping custom code & resources JAR file
[INFO]     Identifying components
[INFO]     Shrinking runtime
[INFO]   
[INFO] Creating media file 'Mac OS X':
[INFO]   Collecting files

[INFO] Installing /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/packaging-3.8.1-sources.jar to /home/cybuilder/.m2/repository/org/cytoscape/distribution/packaging/3.8.1/packaging-3.8.1-sources.jar
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  01:37 min
[INFO] Finished at: 2020-10-05T14:11:44-07:00
[INFO] ------------------------------------------------------------------------


# 5. Copying Cytoscape installers to weekly download page

When completed installer executables can be found in `cytoscape/cytoscape/gui-distribution/packaging/target/media` and compressed builds for Linux and Windows can be found in `cytoscape/cytoscape/gui-distribution/assembly/target`

all of which should be copied to `/var/www/html/cytoscape-builds/Cytoscape-3.8.1/<DATE>`



In [31]:
!mkdir -p /var/www/html/cytoscape-builds/Cytoscape-3.8.1/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/media/* /var/www/html/cytoscape-builds/Cytoscape-3.8.1/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/assembly/target/*.{gz,zip} /var/www/html/cytoscape-builds/Cytoscape-3.8.1/$(date +%Y_%m_%d)


sending incremental file list
Cytoscape_3_8_1_macos.dmg
    279,590,192 100%  217.82MB/s    0:00:01 (xfr#1, to-chk=7/8)
Cytoscape_3_8_1_unix.sh
    277,460,925 100%  243.21MB/s    0:00:01 (xfr#2, to-chk=6/8)
Cytoscape_3_8_1_windows_32bit.exe
    278,970,880 100%  248.41MB/s    0:00:01 (xfr#3, to-chk=5/8)
Cytoscape_3_8_1_windows_64bit.exe
    279,078,400 100%  232.45MB/s    0:00:01 (xfr#4, to-chk=4/8)
md5sums
            254 100%    1.70kB/s    0:00:00 (xfr#5, to-chk=3/8)
output.txt
            789 100%    5.28kB/s    0:00:00 (xfr#6, to-chk=2/8)
sha256sums
            382 100%    2.56kB/s    0:00:00 (xfr#7, to-chk=1/8)
updates.xml
          1,666 100%   11.14kB/s    0:00:00 (xfr#8, to-chk=0/8)

sent 1,115,376,303 bytes  received 168 bytes  247,861,438.00 bytes/sec
total size is 1,115,103,488  speedup is 1.00
sending incremental file list
cytoscape-unix-3.8.1.tar.gz
    274,734,055 100%  304.62MB/s    0:00:00 (xfr#1, to-chk=1/2)
cytoscape-windows-3.8.1.zip
    275,590,551 100%  138.40MB/

## 6. Notarizing on idekerlab-macmini

The install4j generated .dmg must be notarized to run on macOS 10.15 and above. The .dmg will be sent to idekerlab-macmini and submitted for notarization. An email will be sent to William Markuske when the notarization process is complete. As soon as the .dmg is notarized it will work and no further action is needed.

In [32]:
!rsync -av --progress /var/www/html/cytoscape-builds/Cytoscape-3.8.1/$(date +%Y_%m_%d)/*.dmg idekerlab@idekerlab-macmini.ucsd.edu:~/apps_to_notarize/
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizedmg_jing.sh ~/apps_to_notarize/*.dmg Snapshot.$(date +%Y%m%d)'    

building file list ... 
1 file to consider
Cytoscape_3_8_1_macos.dmg
    279,590,192 100%   56.97MB/s    0:00:04 (xfr#1, to-chk=0/1)

sent 279,624,466 bytes  received 46 bytes  43,019,155.69 bytes/sec
total size is 279,590,192  speedup is 1.00
No errors uploading '/Users/idekerlab/apps_to_notarize/Cytoscape_3_8_1_macos.dmg'.
RequestUUID = 1baeec4d-f3f4-4c6d-9984-798d338dec72




You can check the status of the notarization by pasting in the `RequestUUID` value into the following command and running.

In [37]:
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizestatus_jing.sh 1baeec4d-f3f4-4c6d-9984-798d338dec72'

No errors getting notarization info.

          Date: 2020-10-05 21:19:04 +0000
          Hash: ec8e45d3c8a082e743beecf024c4d9f8c8c94db4b1e89d830a71e91b63d2c4df
    LogFileURL: https://osxapps-ssl.itunes.apple.com/itunes-assets/Enigma124/v4/18/44/da/1844da25-b50f-3ab5-860b-baa645754990/developer_log.json?accessKey=1602127481_2485699796868385843_VvjK82d69qfTDAMUbApOXgnwbj1lONbOeCtylMsB8x8SnQLrvhyUkEYTXpym7APnVKTLShgsrEks2kAVNeXgdE7OqNbYl5zFdbuRbxL7WypuIjyE5UxpaG50CHef7IeVZKYac1Cvxi1K%2FZpIzjOZU4GdKHsCjyVG4p%2FJpuY39Y0%3D
   RequestUUID: 1baeec4d-f3f4-4c6d-9984-798d338dec72
        Status: success
   Status Code: 0
Status Message: Package Approved



In [ ]:
updates = [
    'Bug fixes to address memory leaks and race conditions in automation scripts.',
    'Fix for edge tooltips not rendering',
    'New U',
    'Updated apps, including CyBrowser, Analyzer, and CyNDEx-2',
    'Additional Graph Editing Tools'
]
update_list = '\n'.join([' - ' + update for update in updates])
welcome_letter = 'http://www.cytoscape.org/cy3_welcome_letter_v22.pdf'
release_notes = 'http://www.cytoscape.org/release_notes_3_8_1.html'
message = '''Cytoscape {version} is released!

=+=+ =+=+ =+=+ =+=+ =+=+ Hello Cytoscapers! =+=+ =+=+ =+=+ =+=+ =+=+
 
We're very pleased and excited to announce availability of Cytoscape {version}, effective immediately!

To get a copy, surf to our download page (http://cytoscape.org/download), and download and execute the installer for your workstation.

There are many new and exciting features in {version}, including:
{update_list}

Note that with {version}, Windows & Mac systems can run Java versions 8, 9, and 10 without conflicting with Cytoscape -- it installs and uses a private Java 11.

For Linux, you must be sure Java 11 is installed and is first on the PATH or is named via the JAVA_HOME environment variable.

If you find any issues with this release, please let us know by posting to cytoscape-helpdesk@googlegroups.com … we very much value your feedback!

Please check the {version} materials in our Welcome Letter ({welcome_letter}) and the Release Notes ({release_notes}).

Tell all your friends ...

We appreciate your checking this out, and we hope you find it as great to work with as we do!

=+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+ =+=+'''
print(message.format(version='3.8.0', update_list=update_list, 
                     welcome_letter=welcome_letter, release_notes=release_notes))